In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import binary_dilation

from dipy.core.gradients import gradient_table
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti_data, load_nifti, save_nifti
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_trk
from dipy.direction import peaks
from dipy.reconst import shm
from dipy.tracking import utils
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.stopping_criterion import BinaryStoppingCriterion
from dipy.tracking.streamline import Streamlines
from dipy.viz import window, actor, colormap as cmap
from os.path import join as pjoin

from dipy.align import affine_registration, syn_registration
from dipy.align.reslice import reslice
from dipy.core.gradients import gradient_table
from dipy.data import get_fnames, fetch_stanford_tracks
from dipy.data.fetcher import (fetch_mni_template, read_mni_template)
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_tractogram, load_tractogram
from dipy.segment.mask import median_otsu
from dipy.tracking.streamline import transform_streamlines
from dipy.viz import regtools, has_fury, horizon
import os

import nibabel as nib
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import load_tractogram, save_tractogram
from dipy.io.utils import (create_nifti_header, get_reference_info,
                           is_header_compatible)
from dipy.tracking.streamline import select_random_set_of_streamlines
from dipy.tracking.utils import density_map

from dipy.data.fetcher import (fetch_file_formats,
                               get_file_formats)

In [3]:
import os
import subprocess

# Set up FSL environment
fsl_path = "/home/a1/fsl"
os.environ["FSLDIR"] = fsl_path
os.environ["PATH"] += os.pathsep + os.path.join(fsl_path, "bin")
os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"

# Define base directory where the BIDS data is stored
BASE_DIR = "/home/a1/Desktop/Projects/forkx/MDD_nii"
FREE_DIR = "/home/a1/Desktop/Projects/forkx/MDD_nii"
# Path to the list of subjects file
SUBJECT_LIST = "/home/a1/Desktop/Projects/code/mdd_list.txt"

# Read the list of subjects
with open(SUBJECT_LIST, 'r') as file:
    subjects = file.read().splitlines()

# Loop over each subject in the list
for SUBJ in subjects:
    SUBJ_ID = f"sub-{SUBJ}"
    print(f"Processing {SUBJ_ID}")

    # Define subject directory
    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ_ID)

    # Define file paths
    t1_fname = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_brainmask_in_space.nii.gz")
    label_fname = os.path.join(FREE_DIR, f"{SUBJ}","mri", "wm_bin.nii.gz")
    MAT = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_brainmask_to_dwi.mat")
    BVAL = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi.bval")
    BVEC = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi.bvec")
    WM = os.path.join(FREE_DIR, f"{SUBJ}","mri", "wm_in_dwi.nii.gz")
    EDDY_IMAGE = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi_brain.nii.gz")
    OUT=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_b0_brain_in_MNI.nii.gz")
    WM_bin = os.path.join(FREE_DIR, f"{SUBJ}","mri", "wm_bin.nii.gz")
    WM_MIF = os.path.join(FREE_DIR, f"{SUBJ}","mri", "wm.mif")
    B0=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_b0.nii.gz")
    mean_B0=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_b0_mean.nii.gz")
    brain=os.path.join(FREE_DIR, f"{SUBJ}","mri", "brain.mgz")
    brainnii=os.path.join(FREE_DIR, f"{SUBJ}","mri", "brain.nii.gz")

    eddy_OUTPUT=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi_in_MNI.nii.gz")
    B0_in_MNI=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_b0_brain_in_MNI.nii.gz1Warp.nii.gz") 
    TransMAT=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_b0_brain_in_MNI.nii.gz0GenericAffine.mat") 
    MRTIX3_DWI = os.path.join(SUBJ_DIR, 'streamline', f"{SUBJ_ID}_dwi.mif") 
    fOD = os.path.join(SUBJ_DIR, 'streamline', f"{SUBJ_ID}_fod.mif") 
    reponsetxt=os.path.join(SUBJ_DIR, 'streamline', f"{SUBJ_ID}_response_wm.txt") 
    reponsetxt1=os.path.join(SUBJ_DIR, 'streamline', f"{SUBJ_ID}_response_gm.txt") 
    reponsetxt2=os.path.join(SUBJ_DIR, 'streamline', f"{SUBJ_ID}_response_csf.txt") 

    streamlines=os.path.join(SUBJ_DIR, 'streamline', f"{SUBJ_ID}_streamline.tck") 

    flix=os.path.join(SUBJ_DIR, 'streamline', f"{SUBJ_ID}")
    t1_fname = os.path.join(SUBJ_DIR, 'anat', f"{SUBJ_ID}_sub-402_T1w.nii.gz")
    #00#
    convert_cmd = ['mrconvert', EDDY_IMAGE, MRTIX3_DWI, '-fslgrad', BVEC, BVAL,'-force']

    try:
        subprocess.run(convert_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BET: {e}")
        continue
    #211#
    convert_cmd = ['dwi2mask', MRTIX3_DWI, WM_MIF, '-force']
    try:
        subprocess.run(convert_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BET: {e}")
        continue
    

    #33#
    convert_cmd = ['dwi2response', 'dhollander', MRTIX3_DWI,reponsetxt, reponsetxt1,reponsetxt2, '-force']
    try:              
        subprocess.run(convert_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BET: {e}")
        continue
    
    
    #44#
    convert_cmd = ['dwi2fod', 'msmt_csd',  MRTIX3_DWI, reponsetxt ,fOD, '-force']
    try:              
        subprocess.run(convert_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BET: {e}")
        continue


    
    TEM='/home/a1/Desktop/Projects/template/MNI152_T1_2mm_brain.nii.gz'
#44#
    convert_cmd = ['tckgen', fOD, streamlines, '-seed_dynamic',fOD, '-select', '100000' ,'-step', '1', '-angle', '60', '-minlength', '10', '-force']          
    try:
        subprocess.run(convert_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BET: {e}")
        continue


Processing sub-011


mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [100%] uncompressing image "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-011/dwi/sub-011_dwi_brain.nii.gz"[0K
mrconvert: [100%] copying from "/home/a1/D...wi/sub-011_dwi_brain.nii.gz" to "/home/a1/D.../streamline/sub-011_dwi.mif"[0K
dwi2mask: [WARNING] existing output files will be overwritten
dwi2mask: [100%] preloading data for "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-011/streamline/sub-011_dwi.mif"[0K
dwi2mask: [done] computing dwi brain mask[0K
dwi2mask: [done] applying mask cleaning filter[0K
dwi2response: 
dwi2response: Note that this script makes use of commands / algorithms that have relevant articles for citation. Please consult the help page (-help option) for more information.
dwi2response: 
dwi2response: Generated scratch directory: /home/a1/Desktop/dwi2response-tmp-WLKS5B/
dwi2response: Importing DWI data (/home/a1/Desktop/Projects/forkx/MDD_nii/sub-011/streamline/sub-011_dwi.mif)...
dwi2response:

Processing sub-031


mrconvert: [100%] uncompressing image "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-031/dwi/sub-031_dwi_brain.nii.gz"[0K
mrconvert: [100%] copying from "/home/a1/D...wi/sub-031_dwi_brain.nii.gz" to "/home/a1/D.../streamline/sub-031_dwi.mif"[0K
dwi2mask: [WARNING] existing output files will be overwritten
dwi2mask: [100%] preloading data for "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-031/streamline/sub-031_dwi.mif"[0K
dwi2mask: [done] computing dwi brain mask[0K
dwi2mask: [done] applying mask cleaning filter[0K
dwi2response: 
dwi2response: Note that this script makes use of commands / algorithms that have relevant articles for citation. Please consult the help page (-help option) for more information.
dwi2response: 
dwi2response: Generated scratch directory: /home/a1/Desktop/dwi2response-tmp-XKEI1M/
dwi2response: Importing DWI data (/home/a1/Desktop/Projects/forkx/MDD_nii/sub-031/streamline/sub-031_dwi.mif)...
dwi2response: Changing to scratch directory (/home/a1/Desktop/dwi2response-t

Processing sub-034
Error running BET: Command '['mrconvert', '/home/a1/Desktop/Projects/forkx/MDD_nii/sub-034/dwi/sub-034_dwi_brain.nii.gz', '/home/a1/Desktop/Projects/forkx/MDD_nii/sub-034/streamline/sub-034_dwi.mif', '-fslgrad', '/home/a1/Desktop/Projects/forkx/MDD_nii/sub-034/dwi/sub-034_dwi.bvec', '/home/a1/Desktop/Projects/forkx/MDD_nii/sub-034/dwi/sub-034_dwi.bval', '-force']' returned non-zero exit status 1.
Processing sub-036
Error running BET: Command '['mrconvert', '/home/a1/Desktop/Projects/forkx/MDD_nii/sub-036/dwi/sub-036_dwi_brain.nii.gz', '/home/a1/Desktop/Projects/forkx/MDD_nii/sub-036/streamline/sub-036_dwi.mif', '-fslgrad', '/home/a1/Desktop/Projects/forkx/MDD_nii/sub-036/dwi/sub-036_dwi.bvec', '/home/a1/Desktop/Projects/forkx/MDD_nii/sub-036/dwi/sub-036_dwi.bval', '-force']' returned non-zero exit status 1.
Processing sub-040


...

mrconvert: [100%] uncompressing image "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-294/dwi/sub-294_dwi_brain.nii.gz"[0K
mrconvert: [100%] copying from "/home/a1/D...wi/sub-294_dwi_brain.nii.gz" to "/home/a1/D.../streamline/sub-294_dwi.mif"[0K
dwi2mask: [WARNING] existing output files will be overwritten
dwi2mask: [100%] preloading data for "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-294/streamline/sub-294_dwi.mif"[0K
dwi2mask: [done] computing dwi brain mask[0K
dwi2mask: [done] applying mask cleaning filter[0K
dwi2response: 
dwi2response: Note that this script makes use of commands / algorithms that have relevant articles for citation. Please consult the help page (-help option) for more information.
dwi2response: 
dwi2response: Generated scratch directory: /home/a1/Desktop/dwi2response-tmp-T1EOSP/
dwi2response: Importing DWI data (/home/a1/Desktop/Projects/forkx/MDD_nii/sub-294/streamline/sub-294_dwi.mif)...
dwi2response: Changing to scratch directory (/home/a1/Desktop/dwi2response-t

Processing sub-318


mrconvert: [100%] uncompressing image "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-318/dwi/sub-318_dwi_brain.nii.gz"[0K
mrconvert: [100%] copying from "/home/a1/D...wi/sub-318_dwi_brain.nii.gz" to "/home/a1/D.../streamline/sub-318_dwi.mif"[0K
dwi2mask: [WARNING] existing output files will be overwritten
dwi2mask: [100%] preloading data for "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-318/streamline/sub-318_dwi.mif"[0K
dwi2mask: [done] computing dwi brain mask[0K
dwi2mask: [done] applying mask cleaning filter[0K
dwi2response: 
dwi2response: Note that this script makes use of commands / algorithms that have relevant articles for citation. Please consult the help page (-help option) for more information.
dwi2response: 
dwi2response: Generated scratch directory: /home/a1/Desktop/dwi2response-tmp-I0BFFX/
dwi2response: Importing DWI data (/home/a1/Desktop/Projects/forkx/MDD_nii/sub-318/streamline/sub-318_dwi.mif)...
dwi2response: Changing to scratch directory (/home/a1/Desktop/dwi2response-t

Processing sub-329


mrconvert: [100%] uncompressing image "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-329/dwi/sub-329_dwi_brain.nii.gz"[0K
mrconvert: [100%] copying from "/home/a1/D...wi/sub-329_dwi_brain.nii.gz" to "/home/a1/D.../streamline/sub-329_dwi.mif"[0K
dwi2mask: [WARNING] existing output files will be overwritten
dwi2mask: [100%] preloading data for "/home/a1/Desktop/Projects/forkx/MDD_nii/sub-329/streamline/sub-329_dwi.mif"[0K
dwi2mask: [done] computing dwi brain mask[0K
dwi2mask: [done] applying mask cleaning filter[0K
dwi2response: 
dwi2response: Note that this script makes use of commands / algorithms that have relevant articles for citation. Please consult the help page (-help option) for more information.
dwi2response: 
dwi2response: Generated scratch directory: /home/a1/Desktop/dwi2response-tmp-Z3YTM2/
dwi2response: Importing DWI data (/home/a1/Desktop/Projects/forkx/MDD_nii/sub-329/streamline/sub-329_dwi.mif)...
dwi2response: Changing to scratch directory (/home/a1/Desktop/dwi2response-t